In [1]:
!pip install transformers

In [2]:
import pandas as pd
from transformers import pipeline
import math
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
import numpy as np

In [3]:
df = pd.read_excel('Reviews 4.15.19.xlsx')

In [4]:
df.head()

,Employer Name,Review Link,Job Title,Review Date,Review City Name,Review State Name,Review Country Name,Current Job,Length of Employment,Employment Status,...,Work-Life Balance,Culture & Values,Business Outlook,Recommend to Friend,CEO Approval,Pros,Cons,Advice to Sr. Management,Headline,Employer Response
0,General Motors Corporation,http://www.glassdoor.com/Reviews/Employee-Revi...,Senior IT Developer,2019-04-14,Austin,Texas,US,No,More than 3 years,Full-time,...,0,0,,,No Rating,4 weeks of vacation unlimited sick time,work delivery is slow. Lay off culture.,,Good company to work,
1,General Motors Corporation,http://www.glassdoor.com/Reviews/Employee-Revi...,Senior Project Engineer,2019-04-13,Warren,Michigan,US,Yes,More than 5 years,Full-time,...,3,1,Getting Worse,No,Disapprove,"Benefit package is good, including vacation_x0...",Miserable atmosphere since many very talented ...,Get new Executive Leadership team replacing CE...,Hoping To Leave GM Soon,
2,General Motors Corporation,http://www.glassdoor.com/Reviews/Employee-Revi...,,2019-04-12,,,,Yes,,Full-time,...,3,4,,Yes,Approve,exposure to new technologies and learning oppo...,no time or schedule training to learn new tech...,,review,
3,General Motors Corporation,http://www.glassdoor.com/Reviews/Employee-Revi...,,2019-04-12,,,,No,,Full-time,...,4,3,Getting Better,Yes,Approve,Lots of opportunities for career development,Heavy politics not motivating people,,Great company,
4,General Motors Corporation,http://www.glassdoor.com/Reviews/Employee-Revi...,,2019-04-12,,,,No,,Full-time,...,4,5,Getting Better,Yes,Approve,Great roles and fun environment. Would still ...,Intern program focus needs to focus on functio...,Don't give up! Future is autonomous!,Good Employer,


In [5]:
df_test = df[['Pros', 'Cons', 'Headline', 'Overall Satisfaction', 'Advice to Sr. Management']]

In [6]:
df_test.head()

,Pros,Cons,Headline,Overall Satisfaction,Advice to Sr. Management
0,4 weeks of vacation unlimited sick time,work delivery is slow. Lay off culture.,Good company to work,3,
1,"Benefit package is good, including vacation_x0...",Miserable atmosphere since many very talented ...,Hoping To Leave GM Soon,1,Get new Executive Leadership team replacing CE...
2,exposure to new technologies and learning oppo...,no time or schedule training to learn new tech...,review,3,
3,Lots of opportunities for career development,Heavy politics not motivating people,Great company,4,
4,Great roles and fun environment. Would still ...,Intern program focus needs to focus on functio...,Good Employer,4,Don't give up! Future is autonomous!


In [7]:
def remove_nan(x):
    if type(x)==float:
        return ''
    else:
        return x
df_test['Pros'] = df_test['Pros'].map(lambda x: remove_nan(x))
df_test['Cons'] = df_test['Cons'].map(lambda x: remove_nan(x))
df_test['Advice to Sr. Management'] = df_test['Advice to Sr. Management'].map(lambda x: remove_nan(x))

<ipython-input-7-35abf558a1ad>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Pros'] = df_test['Pros'].map(lambda x: remove_nan(x))
<ipython-input-7-35abf558a1ad>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['Cons'] = df_test['Cons'].map(lambda x: remove_nan(x))
<ipython-input-7-35abf558a1ad>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [8]:
heading_list = df_test['Headline'].tolist()
pros_list = df_test['Pros'].tolist()
cons_list = df_test['Cons'].tolist()
advice_list = df_test['Advice to Sr. Management'].tolist()

In [9]:
print(len(cons_list))
print(len(pros_list))
print(len(heading_list))
print(len(advice_list))

434
434
434
434


In [10]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
all_list = {'cons':cons_list, 'pros':pros_list, 'heading':heading_list, 'advice': advice_list}
model = model.to(device)
all_stars = {'cons':[], 'pros':[], 'heading':[], 'advice':[]}
for type_input in ['cons', 'pros', 'heading', 'advice']:
    for i in range(math.ceil(len(cons_list)/100)):
        max_number = (i+1)*100
        if i ==math.ceil(len(cons_list)/100)-1:
            max_number = len(cons_list)
        pt_batch = tokenizer(
            all_list[type_input][i*100:max_number],
            padding=True,
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)
        
        pt_outputs = model(**pt_batch)
        pt_predictions = F.softmax(pt_outputs.logits, dim=-1)
        stars_cuda = torch.argmax(pt_predictions, dim=-1)
        stars = stars_cuda.cpu().numpy().tolist()
        all_stars[type_input]=all_stars[type_input]+stars
        del pt_outputs
        del pt_batch
        del stars_cuda
        del pt_predictions
        if device == 'cuda:0':
            torch.cuda.empty_cache()


In [ ]:
df_test['cons_sentiment'] = all_stars['cons']
df_test['pros_sentiment'] = all_stars['pros']
df_test['heading_sentiment'] = all_stars['heading']
df_test['advice_sentiment'] = all_stars['advice']

In [ ]:
df_test

In [ ]:
predict_heading_sentiment = (df_test['heading_sentiment'].to_numpy()>2.5).tolist()

In [ ]:
true_sentiment = (df_test['Overall Satisfaction'].to_numpy()>2.5).tolist() 

In [ ]:
Accuracy = np.logical_and(true_sentiment, predict_heading_sentiment).mean()
print(f"Accuracy of Sentiment Classification is {Accuracy}")